In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [ ]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_images = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_images = train_df.drop(labels = ["label"],axis = 1) 
train_labels = train_df.label

In [ ]:
train_images.shape

In [ ]:
train_labels.shape

In [ ]:
test_images.head()

In [ ]:
train_images  =train_images/255.0
test_images = test_images/255.0

In [ ]:
train_images

In [ ]:
import matplotlib.pyplot as plt

img = train_images.iloc[0].values.reshape((28,28))
plt.imshow(img,cmap='gray')
plt.title(train_images.iloc[0,0])
plt.axis("off")
plt.show()

In [ ]:
img = train_images.iloc[10].values.reshape((28,28))
plt.imshow(img,cmap='gray')
plt.title(train_images.iloc[0,0])
plt.axis("off")
plt.show()

In [ ]:
train_images=train_images.values.reshape(len(train_images),28,28,1)
test_images=test_images.values.reshape(len(test_images),28,28,1)

In [ ]:
train_images.shape

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
   
    keras.layers.MaxPool2D(pool_size=(2,2)),
    keras.layers.Dropout(0.25),
      
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=256, step=16),
        activation='relu'
    ),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="digit Recognizer")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=5,validation_split=0.1)

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.fit(train_images, train_labels, epochs=15, validation_split=0.1, initial_epoch=3)

In [ ]:
predictions = model.predict(test_images).argmax(axis = 1)
predictions
submission = pd.DataFrame({'ImageId':np.arange(1, len(predictions)+1), 'Label':predictions})
submission.to_csv('submission.csv', index=False)